In [2]:
%matplotlib notebook
from rfsoc_rfdc.rfsoc_overlay import RFSoCOverlay
from rfsoc_rfdc.overlay_task import OverlayTask
from rfsoc_rfdc.overlay_task import BlinkLedTask

from rfsoc_rfdc.transmitter.single_ch_tx_task import SingleChTxTask
from rfsoc_rfdc.receiver.single_ch_rx_task import SingleChRxTask

from rfsoc_rfdc.transmitter.multi_ch_tx_task import MultiChTxTask
from rfsoc_rfdc.receiver.multi_ch_rx_task import MultiChRxTask

from rfsoc_rfdc.rfdc_task import RfdcTask 
from rfsoc_rfdc.cmac_task import CmacTask
from rfsoc_rfdc.rfdc_config import ZCU216_CONFIG

import sys
import os
import time

In [3]:
ol = RFSoCOverlay(path_to_bitstream="./rfsoc_rfdc/bitstream/rfsoc_rfdc_v38.bit")
NEW_CONFIG = {
    "RefClockForPLL": 500.0,
    "DACSampleRate": 2500.0,
    "DACInterpolationRate": 2,
    "DACNCO": 625.0,
    "ADCSampleRate": 2500.0,
    "ADCInterpolationRate": 2,
    "ADCNCO": -625.0
}
ZCU216_CONFIG.update(NEW_CONFIG)

adc_datapath/t226/data_mover_ctrl
adc_datapath/t226/fifo_count
axi_gpio_led
dac_datapath/t230/data_mover_ctrl
dac_datapath/t230/fifo_count
usp_rf_data_converter
zynq_ultra_ps_e


In [4]:
rfdc_t = RfdcTask(ol)
led_t = BlinkLedTask(ol)
tx_t = SingleChTxTask(ol, "./wave_files/Tx.npy")
rx_t = SingleChRxTask(ol)

# for task in [rfdc_t]:
#     task.start()
#     task.join()

parallel_task = [led_t, tx_t, rx_t]
for task in parallel_task:
    task.start()

time.sleep(5)

for task in parallel_task:
    task.stop()

FigureWidget({
    'data': [],
    'layout': {'template': '...',
               'title': {'text': 'Complex Sig…

FigureWidget({
    'data': [{'name': 'Magnitude', 'type': 'scattergl', 'uid': '7a540ec9-df48-456d-88f9-7e65ee9…

2024-07-29 14:36:27,326 - root - INFO - Configuring LMK chip using file LMK04828_250.0.txt at 250.0 MHz
2024-07-29 14:36:27,329 - root - INFO - Configuring LMX chip using file LMX2594_500.0.txt at 500.0 MHz
2024-07-29 14:36:29,771 - root - INFO - DAC tile 1 (229) is fully powered up!
2024-07-29 14:36:30,820 - root - INFO - DAC tile 2 (230) is fully powered up!
2024-07-29 14:36:30,823 - root - INFO - DAC tile 0 DAC block 0 is NOT enabled!
2024-07-29 14:36:30,826 - root - INFO - DAC tile 0 DAC block 1 is NOT enabled!
2024-07-29 14:36:30,828 - root - INFO - DAC tile 0 DAC block 2 is NOT enabled!
2024-07-29 14:36:30,831 - root - INFO - DAC tile 0 DAC block 3 is NOT enabled!
2024-07-29 14:36:30,923 - root - INFO - DAC tile 1 DAC block 0 is enabled!
2024-07-29 14:36:30,926 - root - INFO - DAC tile 1 DAC block 1 is NOT enabled!
2024-07-29 14:36:30,928 - root - INFO - DAC tile 1 DAC block 2 is NOT enabled!
2024-07-29 14:36:30,931 - root - INFO - DAC tile 1 DAC block 3 is NOT enabled!
2024-07-2